In [4]:
print("Starting script...")

from modelling import *
from modelling import GRU

import os
from pathlib import Path
import datetime
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset

Starting script...


In [5]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("Device: ", device)

Device:  cuda


In [6]:
HABROK = bool(0)                  # set to True if using HABROK; it will print
                                  # all stdout to a .txt file to log progress
CITY_NAME = "Utrecht"
BASE_DIR = Path.cwd()
MODEL_PATH = BASE_DIR / "results" / "models"
MINMAX_PATH = BASE_DIR.parent / "data" / "data_combined" / CITY_NAME.lower() / "contaminant_minmax.csv"

print("BASE_DIR: ", BASE_DIR)
print("MODEL_PATH: ", MODEL_PATH)
print("MINMAX_PATH: ", MINMAX_PATH)

torch.manual_seed(34)             # set seed for reproducibility

N_HOURS_U = 72                    # number of hours to use for input
N_HOURS_Y = 24                    # number of hours to predict
N_HOURS_STEP = 24                 # "sampling rate" in hours of the data; e.g. 24 
                                  # means sample an I/O-pair every 24 hours
                                  # the contaminants and meteorological vars
CONTAMINANTS = ['NO2', 'O3'] # 'PM10', 'PM25']
COMPONENTS = ['NO2', 'O3', 'PM10', 'PM25', 'SQ', 'WD', 'Wvh', 'dewP', 'p', 'temp']

BASE_DIR:  /home/nick/bachelor-project/forecasting_smog_DL_GNN/src
MODEL_PATH:  /home/nick/bachelor-project/forecasting_smog_DL_GNN/src/results/models
MINMAX_PATH:  /home/nick/bachelor-project/forecasting_smog_DL_GNN/data/data_combined/utrecht/contaminant_minmax.csv


In [19]:
hp_gru = {
    'n_hours_u' : N_HOURS_U,
    'n_hours_y' : N_HOURS_Y,

    'model_class' : GRU, # changed to GRU
    'input_units' : 8, #train_dataset.__n_features_in__(),
    'hidden_layers' : 6,
    'hidden_units' : 128,
    # 'branches' : 2,  # predicting only no2 and o3
    'output_units' : 2, #train_dataset.__n_features_out__(),

    'Optimizer' : torch.optim.Adam,
    'lr_shared' : 1e-3,
    'scheduler' : torch.optim.lr_scheduler.ReduceLROnPlateau,
    'scheduler_kwargs' : {'mode' : 'min',
                          'factor' : 0.1,
                          'patience' : 3,
                          'cooldown' : 8,
                          'verbose' : True},
    'w_decay' : 1e-5,
    'loss_fn' : torch.nn.MSELoss(),

    'epochs' : 5000,
    'early_stopper' : EarlyStopper,
    'patience' : 15,
    'batch_sz' : 16,
    'k_folds' : 5,
}

In [25]:
def test(model, loss_fn, test_loader, denorm=False, path=None) -> float:
    model.eval()
    test_loss = np.float64(0)
    
    # Ensure the model is on the correct device
    device = next(model.parameters()).device

    with torch.no_grad():
        for batch_test_u, batch_test_y in test_loader:
            batch_test_u = batch_test_u.to(device)
            batch_test_y = batch_test_y.to(device)
            
            pred = model(batch_test_u)
            if denorm:
                pred = denormalise(pred, path)
                batch_test_y = denormalise(batch_test_y, path)
            
            test_loss += loss_fn(pred, batch_test_y).item()

    return test_loss / len(test_loader)


In [26]:
def test_separately(
    model,
    loss_fn,
    test_loader,
    denorm: bool = False,
    path: str = None,
    components=["NO2", "O3", "PM10", "PM25"],
):
    """
    Evaluates on test set and returns test loss

    :param model: model to evaluate, must be some PyTorch type model
    :param loss_fn: loss function to use, PyTorch defined, or PyTorch inherited
    :param test_loader: DataLoader to get batches from
    :param denorm: whether to denormalise the data before calculating loss
    :param path: path to the file containing the minmax values for the data
    :return: dictionary with contaminant names as keys and losses as values
    """
    model.eval()
    device = next(model.parameters()).device
    test_losses = [np.float64(0) for _ in components]

    with torch.no_grad():
        for batch_test_u, batch_test_y in test_loader:
            pred = model(batch_test_u.to(device))
            if denorm:
                pred = denormalise(pred, path)
                batch_test_y = denormalise(batch_test_y.to(device), path)

            for comp in range(len(components)):
                test_losses[comp] += loss_fn(
                    pred[:, :, comp], batch_test_y[:, :, comp]
                ).item()

    for comp in range(len(components)):
        test_losses[comp] /= len(test_loader)
    return {comp: loss for comp, loss in zip(components, test_losses)}


# 3 different GRUs on each city

## Utrecht

In [20]:
train_input_frames = get_dataframes('train', 'u', CITY_NAME)
train_output_frames = get_dataframes('train', 'y', CITY_NAME)

val_input_frames = get_dataframes('val', 'u', CITY_NAME)
val_output_frames = get_dataframes('val', 'y', CITY_NAME)

test_input_frames = get_dataframes('test', 'u', CITY_NAME)
test_output_frames = get_dataframes('test', 'y', CITY_NAME)

print("Successfully loaded data")

Successfully loaded data


In [31]:
train_dataset = TimeSeriesDataset(
    train_input_frames,  # list of input training dataframes
    train_output_frames, # list of output training dataframes
    5,                   # number of dataframes put in for both
                         # (basically len(train_input_frames) and
                         # len(train_output_frames) must be equal)
    N_HOURS_U,           # number of hours of input data
    N_HOURS_Y,           # number of hours of output data
    N_HOURS_STEP,        # number of hours between each input/output pair
)
val_dataset = TimeSeriesDataset(
    val_input_frames,    # etc.
    val_output_frames,
    3,
    N_HOURS_U,
    N_HOURS_Y,
    N_HOURS_STEP,
)
test_dataset = TimeSeriesDataset(
    test_input_frames,
    test_output_frames,
    3,
    N_HOURS_U,
    N_HOURS_Y,
    N_HOURS_STEP,
)

del train_input_frames, train_output_frames
del val_input_frames, val_output_frames
del test_input_frames, test_output_frames

In [21]:
with PrintManager('.', 'a', HABROK):
    print("\nPrinting model:")
    utrecht_model = GRU(hp_gru['n_hours_u'],
                 hp_gru['n_hours_y'],
                 hp_gru['input_units'],
                 hp_gru['hidden_layers'],
                 hp_gru['hidden_units'], 
                #  hp['branches'],
                 hp_gru['output_units'])
    print(utrecht_model)


Printing model:
GRU(
  (gru): GRU(8, 128, num_layers=6, batch_first=True)
  (dense): Linear(in_features=128, out_features=2, bias=True)
)


In [22]:
utrecht_model.load_state_dict(torch.load(MODEL_PATH / "model_GRU_utrecht.pth"))
utrecht_model.to(device)

GRU(
  (gru): GRU(8, 128, num_layers=6, batch_first=True)
  (dense): Linear(in_features=128, out_features=2, bias=True)
)

In [34]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
stdout_location = f'results/grid_search_exe_s/exe_of_GRU_at_{current_time}.txt'

In [36]:
train_loader = DataLoader(train_dataset, batch_size = hp_gru['batch_sz'], shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = hp_gru['batch_sz'], shuffle = False) 
                                            
#                                         # Train the final model on the full training set,
#                                         # save the final model, and save the losses for plotting
with PrintManager('.', 'a', HABROK):
    print("\nTraining on full training set...")
    model_final, train_losses, val_losses = \
        train(hp_gru, train_loader, val_loader, True)
    torch.save(model_final.state_dict(), f'{MODEL_PATH}/model_GRU.pth')

df_losses = pd.DataFrame({'L_train': train_losses, 'L_val': val_losses})
df_losses.to_csv(f'{os.path.join(os.getcwd(), "results/final_losses")}/losses_GRU_at_{current_time}_{CITY_NAME}.csv', 
                 sep = ';', decimal = '.', encoding = 'utf-8')


Training on full training set...
Epoch: 1 	Ltrain: 0.019557 	Lval: 0.010785
Epoch: 5 	Ltrain: 0.006436 	Lval: 0.005834
Epoch: 10 	Ltrain: 0.005904 	Lval: 0.004261
Epoch: 15 	Ltrain: 0.005371 	Lval: 0.004182
Epoch 00020: reducing learning rate of group 0 to 1.0000e-04.
Epoch: 20 	Ltrain: 0.005230 	Lval: 0.004690
Epoch: 25 	Ltrain: 0.004979 	Lval: 0.003930
Epoch: 30 	Ltrain: 0.004962 	Lval: 0.003826
Epoch 00032: reducing learning rate of group 0 to 1.0000e-05.
Epoch: 35 	Ltrain: 0.004928 	Lval: 0.004024
EarlyStopper: stopping at epoch 36 with best_val_loss = 0.003820



In [37]:
test_loader = DataLoader(test_dataset, batch_size = hp_gru['batch_sz'], shuffle = False) 
loss_fn = nn.MSELoss()  # Instantiate the loss function
test_error = test(utrecht_model, loss_fn, test_loader)

with PrintManager('.', 'a', HABROK):
    print()
    print("Testing MSE:", test_error)


Testing MSE: 0.0031511762257044515


In [38]:
print_dict_vertically_root(
    test_separately(utrecht_model, nn.MSELoss(), test_loader, True, MINMAX_PATH, components=["NO2", "O3"])
)

NO2: 6.949684371916662
O3 : 8.454666984221609
